In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('D:\dev\project\Football-Match-Prediction\data\processed\la_liga_data.csv')
data['Time'] = pd.to_datetime(data['Time'])
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5520 entries, 0 to 5519
Data columns (total 165 columns):
 #    Column                Dtype         
---   ------                -----         
 0    Time                  datetime64[ns]
 1    Comp                  object        
 2    Round                 int64         
 3    Day                   int64         
 4    Venue                 int64         
 5    Result                object        
 6    GF                    float64       
 7    GA                    float64       
 8    Opponent              object        
 9    xG                    float64       
 10   xGA                   float64       
 11   Captain               object        
 12   Formation             object        
 13   Opp Formation         object        
 14   Referee               object        
 15   Standard__Gls         int64         
 16   Standard__Sh          float64       
 17   Standard__SoT         float64       
 18   Standard__SoT%        floa

In [2]:
data.head()

,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,...,Performance__OG,Performance__Recov,Aerial Duels__Won,Aerial Duels__Lost,Aerial Duels__Won%,Season,Team,Points,Poss,GD
0,2017-08-18 20:15:00,La Liga,1,5,0,L,0.0,1.0,Leganes,1.1,...,0.0,60.0,16.0,30.0,34.8,2017,Alaves,0,47.0,-1.0
1,2017-08-18 20:15:00,La Liga,1,5,1,W,1.0,0.0,Alaves,1.3,...,0.0,55.0,30.0,16.0,65.2,2017,Leganes,3,53.0,1.0
2,2017-08-18 22:15:00,La Liga,1,5,0,L,0.0,1.0,Valencia,0.2,...,0.0,42.0,6.0,8.0,42.9,2017,Las Palmas,0,53.0,-1.0
3,2017-08-18 22:15:00,La Liga,1,5,1,W,1.0,0.0,Las Palmas,1.4,...,0.0,54.0,8.0,6.0,57.1,2017,Valencia,3,47.0,1.0
4,2017-08-19 18:15:00,La Liga,1,6,1,L,2.0,3.0,Real Sociedad,1.8,...,0.0,48.0,11.0,12.0,47.8,2017,Celta Vigo,0,53.0,-1.0


In [2]:
data.drop(columns=['Comp', 'Day', 'Captain', 'Formation', 'Opp Formation', 'Referee'], inplace=True)

In [3]:
for col in data.select_dtypes(include=[np.number]).columns:
    data[col] = data[col].fillna(data.groupby('Team')[col].transform('mean'))
    # df2[cols] = df2[cols].fillna(df2.groupby('Team')[cols].mean()) 

#### Get current form features

In [4]:
current_form_features = pd.read_csv('D:/dev/project/Football-Match-Prediction/src/eda/eda_data/selected/all_feature_names.csv')#.drop(axis = 0, labels=['Performance__GA', 'Standard__Gls'], inplace=False)   
window_size = 5
current_form_features = set(current_form_features['Feature_Name'].values)
current_form_features.remove('Performance__GA')
current_form_features.remove('Standard__Gls')
current_form_features = list(current_form_features)
current_form_features

['GCA Types__PassLive',
 'Challenges__Lost',
 'Standard__G/SoT',
 'Standard__SoT',
 'GCA Types__TO',
 'Err',
 'Standard__SoT%',
 'GCA Types__Sh',
 'Performance__CS',
 'Performance__Save%',
 'Performance__PSxG',
 'GCA Types__GCA',
 'Clr',
 'Standard__G/Sh']

In [8]:
# Sort the data by Team, Season, Round, and Time
data = data.sort_values(['Team', 'Season', 'Round', 'Time'])
train_data = data[['Time', 'Season', 'Venue', 'Round', 'Team', 'Opponent', 'GF', 'GA', 'Result']].copy()
# Calculate rolling averages for the last 5 matches
for feature in current_form_features:
    if feature != 'Performance__CS':
        train_data[f"{feature}_avg_last5"] = data.groupby('Team')[feature].transform(
            lambda x: x.shift().rolling(window=window_size, min_periods=1).mean()
        )
    else:
        train_data[f"{feature}_avg_last5"] = data.groupby('Team')[feature].transform(
            lambda x: x.shift().rolling(window=window_size, min_periods=1).sum()
        )

# Calculate the current points
train_data['CurrentPoints'] = data.groupby(['Team', 'Season'])['Points'].transform(
    lambda x: x.shift().cumsum()
).fillna(0)

# Calculate total points per team and season
team_season_points = data.groupby(['Team', 'Season'])['Points'].sum().reset_index()

# Sort values to ensure correct shift
team_season_points.sort_values(['Team', 'Season'], inplace=True)

# Shift the points to get last season's points
team_season_points['LastSeasonPoints'] = team_season_points.groupby('Team')['Points'].shift(1).fillna(0)

# Merge the last season points back to train_data
train_data = train_data.merge(team_season_points[['Team', 'Season', 'LastSeasonPoints']], on=['Team', 'Season'], how='left')

In [14]:
train_data.sort_values(['Team', 'Season', 'Round', 'Time'], inplace=True)
# pd.set_option('display.max_rows', None)
# train_data[['Team', 'Season', 'Round', 'CurrentPoints', 'LastSeasonPoints']][train_data['Team'] == 'Leganes'].head(80)

In [15]:
prematch_cols = ['Time', 'Round', 'Venue', 'Season']
# train_data = data[prematch_cols + ['Team', 'Opponent', 'Result', 'GF', 'GA']].sort_values(by=['Season', 'Round', 'Time'])
# Rename columns to include 'HomeTeam_' prefix where Venue == 1
df_team = train_data[train_data['Venue'] == 1].copy()
df_team['HomeTeam'] = df_team['Team']
df_team.drop(columns=['Team'], inplace=True)
df_team['AwayTeam'] = df_team['Opponent']
df_team.drop(columns=['Opponent'], inplace=True)
df_team = df_team.rename(columns=lambda x: f"HomeTeam_{x}" if x not in (prematch_cols + ['HomeTeam', 'AwayTeam']) else x)
df_team.drop(columns=['Venue'], inplace=True)
df_opponent = train_data[train_data['Venue'] == 0].copy()
df_opponent['AwayTeam'] = df_opponent['Team']
df_opponent = df_opponent.drop(columns=['Team'])
df_opponent['HomeTeam'] = df_opponent['Opponent']
df_opponent = df_opponent.drop(columns=['Opponent'])
df_opponent = df_opponent.rename(columns=lambda x: f"AwayTeam_{x}" if x not in (prematch_cols + ['HomeTeam', 'AwayTeam']) else x)
df_opponent.drop(columns=['Venue'], inplace=True)   
# Merge the two dataframes
prematch_cols.remove('Venue')
train_data_merged = pd.merge(df_team, df_opponent, on=(prematch_cols + ['HomeTeam', 'AwayTeam']))
train_data_merged.drop(columns=['HomeTeam_GA', 'AwayTeam_GA', 'AwayTeam_Result'], inplace=True)

In [16]:
train_data_merged = train_data_merged.sort_values(['Season', 'Round', 'Time']).reset_index(drop=True)

In [17]:
train_data_merged.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2760 entries, 0 to 2759
Data columns (total 40 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Time                                    2760 non-null   datetime64[ns]
 1   Season                                  2760 non-null   int64         
 2   Round                                   2760 non-null   int64         
 3   HomeTeam_GF                             2760 non-null   float64       
 4   HomeTeam_Result                         2760 non-null   object        
 5   HomeTeam_GCA Types__PassLive_avg_last5  2746 non-null   float64       
 6   HomeTeam_Challenges__Lost_avg_last5     2746 non-null   float64       
 7   HomeTeam_Standard__G/SoT_avg_last5      2746 non-null   float64       
 8   HomeTeam_Standard__SoT_avg_last5        2746 non-null   float64       
 9   HomeTeam_GCA Types__TO_avg_last5        2746 non-nul

In [18]:
train_data.sort_values('Time', inplace=True)

# Initialize a list to store head-to-head stats
h2h_stats = []

# Iterate over each match in df_merged
for index, row in train_data_merged.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    match_time = row['Time']
    
    # Filter past matches between the two teams
    past_matches = train_data[
        (((train_data['Team'] == home_team) & (train_data['Opponent'] == away_team)) |
         ((train_data['Team'] == away_team) & (train_data['Opponent'] == home_team))) &
        (train_data['Time'] < match_time)
    ]
    
    total_matches = len(past_matches)
    if total_matches == 0:
        home_wins = away_wins = draws = home_goals = away_goals = 0
    else:
        # Results from the perspective of the home team
        home_wins = ((past_matches['Team'] == home_team) & (past_matches['Result'] == 'W')).sum()
        away_wins = ((past_matches['Team'] == home_team) & (past_matches['Result'] == 'L')).sum()
        draws = (past_matches['Result'] == 'D').sum()
        
        # Goals scored by each team
        home_goals = past_matches.loc[past_matches['Team'] == home_team, 'GF'].sum() + \
                     past_matches.loc[past_matches['Opponent'] == home_team, 'GA'].sum()
        away_goals = past_matches.loc[past_matches['Team'] == away_team, 'GF'].sum() + \
                     past_matches.loc[past_matches['Opponent'] == away_team, 'GA'].sum()
    
    h2h_stats.append({
        'H2H_Total_Matches': total_matches,
        'H2H_Home_Wins': home_wins,
        'H2H_Away_Wins': away_wins,
        'H2H_Draws': draws,
        'H2H_Home_Goals': home_goals,
        'H2H_Away_Goals': away_goals,
    })

# Convert the list to a DataFrame
h2h_stats_df = pd.DataFrame(h2h_stats)

# Merge the head-to-head stats with the original dataframe
df_merged = pd.concat([train_data_merged.reset_index(drop=True), h2h_stats_df], axis=1)

In [19]:
df_merged.sort_values('Time', inplace=True)
df_merged.head()

,Time,Season,Round,HomeTeam_GF,HomeTeam_Result,HomeTeam_GCA Types__PassLive_avg_last5,HomeTeam_Challenges__Lost_avg_last5,HomeTeam_Standard__G/SoT_avg_last5,HomeTeam_Standard__SoT_avg_last5,HomeTeam_GCA Types__TO_avg_last5,...,AwayTeam_Clr_avg_last5,AwayTeam_Standard__G/Sh_avg_last5,AwayTeam_CurrentPoints,AwayTeam_LastSeasonPoints,H2H_Total_Matches,H2H_Home_Wins,H2H_Away_Wins,H2H_Draws,H2H_Home_Goals,H2H_Away_Goals
0,2017-08-18 20:15:00,2017,1,1.0,W,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0,0,0,0,0.0,0.0
1,2017-08-18 22:15:00,2017,1,1.0,W,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0,0,0,0,0.0,0.0
2,2017-08-19 18:15:00,2017,1,2.0,L,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0,0,0,0,0.0,0.0
3,2017-08-19 20:15:00,2017,1,2.0,D,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0,0,0,0,0.0,0.0
4,2017-08-19 22:15:00,2017,1,1.0,D,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0,0,0,0,0.0,0.0


In [20]:
df_merged.sort_values('Time', inplace=True)
df_merged.to_csv('D:/dev/project/Football-Match-Prediction/data/processed/df_merged.csv', index=False)

In [12]:
# import pandas as pd

# # Ensure 'Time' is datetime and sort the DataFrame chronologically
# train_data_merged['Time'] = pd.to_datetime(train_data_merged['Time'])
# train_data_merged.sort_values('Time', inplace=True)

# # Create a unique identifier for each team pair
# train_data_merged['TeamPair'] = train_data_merged.apply(lambda x: '_'.join(sorted([x['HomeTeam'], x['AwayTeam']])), axis=1)

# # Sort by 'TeamPair' and 'Time' to prepare for cumulative calculations
# train_data_merged.sort_values(['TeamPair', 'Time'], inplace=True)
# train_data_merged.head()


In [13]:
# # Initialize the head-to-head features
# head_to_head_features = [
#     'HeadToHead_Matches',
#     'HeadToHead_HomeWins',
#     'HeadToHead_AwayWins',
#     'HeadToHead_Draws',
#     'HeadToHead_HomeGoals',
#     'HeadToHead_AwayGoals'
# ]

# for feature in head_to_head_features:
#     train_data_merged[feature] = 0

# # Calculate cumulative head-to-head statistics excluding current match
# train_data_merged['HeadToHead_Matches'] = train_data_merged.groupby('TeamPair').cumcount()

# # Home Wins
# train_data_merged['HeadToHead_HomeWins'] = train_data_merged.groupby('TeamPair')['HomeTeam_Result'].transform(lambda x: (x == 'W').cumsum()).shift(1).fillna(0)

# # Away Wins
# train_data_merged['HeadToHead_AwayWins'] = train_data_merged.groupby('TeamPair')['HomeTeam_Result'].transform(lambda x: (x == 'L').cumsum()).shift(1).fillna(0)

# # Draws
# train_data_merged['HeadToHead_Draws'] = train_data_merged.groupby('TeamPair')['HomeTeam_Result'].transform(lambda x: (x == 'D').cumsum()).shift(1).fillna(0)

# # Cumulative Goals
# train_data_merged['HeadToHead_HomeGoals'] = train_data_merged.groupby('TeamPair')['HomeTeam_GF'].transform('cumsum').shift(1).fillna(0)
# train_data_merged['HeadToHead_AwayGoals'] = train_data_merged.groupby('TeamPair')['AwayTeam_GF'].transform('cumsum').shift(1).fillna(0)

In [14]:
pd.set_option('display.max_columns', None)

train_data_merged.sort_values('Time', inplace=True)
train_data_merged.head()

,Time,Season,Round,HomeTeam_GF,HomeTeam_Result,HomeTeam_Err_avg_last5,HomeTeam_Performance__PSxG_avg_last5,HomeTeam_GCA Types__Sh_avg_last5,HomeTeam_GCA Types__PassLive_avg_last5,HomeTeam_Standard__SoT%_avg_last5,HomeTeam_Performance__CS_avg_last5,HomeTeam_Standard__SoT_avg_last5,HomeTeam_Standard__G/Sh_avg_last5,HomeTeam_Standard__G/SoT_avg_last5,HomeTeam_Performance__Save%_avg_last5,HomeTeam_GCA Types__GCA_avg_last5,HomeTeam_Challenges__Lost_avg_last5,HomeTeam_Clr_avg_last5,HomeTeam_GCA Types__TO_avg_last5,HomeTeam_CurrentPoints,HomeTeam,AwayTeam,AwayTeam_GF,AwayTeam_Err_avg_last5,AwayTeam_Performance__PSxG_avg_last5,AwayTeam_GCA Types__Sh_avg_last5,AwayTeam_GCA Types__PassLive_avg_last5,AwayTeam_Standard__SoT%_avg_last5,AwayTeam_Performance__CS_avg_last5,AwayTeam_Standard__SoT_avg_last5,AwayTeam_Standard__G/Sh_avg_last5,AwayTeam_Standard__G/SoT_avg_last5,AwayTeam_Performance__Save%_avg_last5,AwayTeam_GCA Types__GCA_avg_last5,AwayTeam_Challenges__Lost_avg_last5,AwayTeam_Clr_avg_last5,AwayTeam_GCA Types__TO_avg_last5,AwayTeam_CurrentPoints
0,2017-08-18 20:15:00,2017,1,1.0,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Leganes,Alaves,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2017-08-18 22:15:00,2017,1,1.0,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Valencia,Las Palmas,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2017-08-19 18:15:00,2017,1,2.0,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Celta Vigo,Real Sociedad,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2017-08-19 20:15:00,2017,1,2.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Girona,Atletico Madrid,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,2017-08-19 22:15:00,2017,1,1.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Sevilla,Espanyol,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
